> # Instruções: 
> 1) Com [este](https://drive.google.com/file/d/1egeGRqTj_5-q8STWNO3MnWJKSafqmAZT/view?usp=sharing) arquivo, construir um objeto de Serie cronológica com o índice igual ao ano dos jogos olímpicos 
> 2) Separar os nomes próprios de sobrenomes em duas colunas distintas usando vírgula como separador.
> 3) Obter medidas resumidas do conjunto de dados. Qual país ganhou mais medalhas?
> 4) Construir uma tabela mostrando quantas medalhas foram ganhas pelos homens no total em cada ano em que o evento foi realizado.


In [2]:
import pandas as pd

In [41]:
df = pd.read_csv("../Data/Summer-Olympic-medals-1976-to-2008.csv", encoding='latin-1') # Abrindo o arquivo no encode necessário
df = df.dropna() # Eliminando linhas com "not a number"
df.reset_index(inplace=True, drop=True) # Garantindo que os indexes estarão em ordem
df['Year'] = df['Year'].astype(int) # Garantindo que os anos estão como números inteiros
df.head()

,City,Year,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal
0,Montreal,1976,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver
1,Montreal,1976,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze
2,Montreal,1976,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold
3,Montreal,1976,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver
4,Montreal,1976,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze


In [42]:
# Colocando os anos como index e eliminando a coluna "Year"
df.set_index("Year", drop=True, inplace=True)
df.index.name = None
df.head()

,City,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal
1976,Montreal,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver
1976,Montreal,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze
1976,Montreal,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold
1976,Montreal,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver
1976,Montreal,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze


In [43]:
# Separando o nome do sobrenome
df[['last_name','first_name']] = df['Athlete'].str.split(',', n=1, expand = True)
df.head()

,City,Sport,Discipline,Event,Athlete,Gender,Country_Code,Country,Event_gender,Medal,last_name,first_name
1976,Montreal,Aquatics,Diving,3m springboard,"KÖHLER, Christa",Women,GDR,East Germany,W,Silver,KÖHLER,Christa
1976,Montreal,Aquatics,Diving,3m springboard,"KOSENKOV, Aleksandr",Men,URS,Soviet Union,M,Bronze,KOSENKOV,Aleksandr
1976,Montreal,Aquatics,Diving,3m springboard,"BOGGS, Philip George",Men,USA,United States,M,Gold,BOGGS,Philip George
1976,Montreal,Aquatics,Diving,3m springboard,"CAGNOTTO, Giorgio Franco",Men,ITA,Italy,M,Silver,CAGNOTTO,Giorgio Franco
1976,Montreal,Aquatics,Diving,10m platform,"WILSON, Deborah Keplar",Women,USA,United States,W,Bronze,WILSON,Deborah Keplar


In [52]:
# Calculando a quantidade de medalhas por país, no total, e classificando a lista em ordem decrescente
df_country  = df.groupby('Country')['Medal'].count().sort_values(ascending=False)
# Exibindo o primeiro item da lista
print(df_country[:1:])

Country
United States    1992
Name: Medal, dtype: int64


In [62]:
# Calculando a contagem de medalhas para cada país em cada ano, e transformando uma tabela "wide"
# A saída é uma tabela com index = Ano, e um país em cada coluna. Nos valores, a contagem de medalhas.
medal_count = df.groupby([df.index, 'Country'])['Medal'].count().unstack(fill_value=0)
# Encontrando o país com mais medalhas em cada linha
best_country = medal_count.idxmax(axis=1)
# Criando um DF com essa configuração (index=ano, coluna1 = País, coluna2 = # de medalhas)
df_medals = pd.DataFrame({'year': best_country.index, 'country': best_country.values, 'medal_count': medal_count.max(axis=1).values})
# Exibindo o resultado
print(df_medals)

   year        country  medal_count
0  1976   Soviet Union          285
1  1980   Soviet Union          442
2  1984  United States          333
3  1988   Soviet Union          294
4  1992  United States          224
5  1996  United States          260
6  2000  United States          248
7  2004  United States          264
8  2008  United States          315


In [69]:
# Criando a tabela de medalhas por ano, considerando apenas homens
df_man = df[df['Gender'] == 'Men']
# Encontrando a quantidade de medalhas por ano
man_medals  = df_man.groupby(df_man.index)['Medal'].count()
print(man_medals)

1976     924
1980     958
1984     973
1988    1011
1992    1105
1996    1082
2000    1126
2004    1099
2008    1110
Name: Medal, dtype: int64
